## Creación del modelo

### Carga y limpieza

In [1]:
import pandas as pd

# Cargar ambas tablas
sales_final = pd.read_csv(r"C:\Users\pablo\OneDrive\Documentos\HENRY\Proyecto\Inventory\Limpios\SalesFINAL12312016_limpio.csv")
purchase_data = pd.read_csv(r"C:\Users\pablo\OneDrive\Documentos\HENRY\Proyecto\Inventory\Limpios\PurchasesFINAL_limpio.csv")


In [2]:
# Agregamos la columna month a sales_final
sales_final['month'] = pd.to_datetime(sales_final['sales_date']).dt.month

_Asignamos la columna "purchase_price" desde la base de datos "PurchasesFINAL_limpio.csv"_

In [3]:
# Crear diccionario de precios promedio por Brand
price_map = purchase_data.groupby('brand')['purchase_price'].mean().to_dict()

# Asignar a SalesFinal sin merge
sales_final['purchase_price'] = sales_final['brand'].map(price_map)

In [4]:
# Crear diccionario de precios promedio por Brand
purchase_map = purchase_data.groupby('brand')['quantity'].mean().to_dict()

# Asignar a SalesFinal sin merge
sales_final['quantity'] = sales_final['brand'].map(purchase_map)

In [5]:
# Selección de variables relevantes
features = sales_final[['sales_quantity', 'sales_dollars', 'purchase_price', 'volume', 'store']]

Utilizando 'sales_quantity' como variable objetivo relizamos los cortes para hacer las clasificaciones

In [6]:
# Crear categorías de rotación usando percentiles
sales_final['rotacion_categoria'] = pd.cut(
    sales_final['sales_quantity'],
    bins=[-1, 10, 100, sales_final['sales_quantity'].max()],
    labels=['Baja', 'Media', 'Alta']
)

In [7]:
def clasificar_rotacion(row):
    if row['quantity'] > 3 * row['sales_quantity']:
        return 'Sobrestock'
    elif row['sales_quantity'] < 10:
        return 'Baja'
    elif row['sales_quantity'] < 100:
        return 'Media'
    else:
        return 'Alta'

sales_final['rotacion_categoria'] = sales_final.apply(clasificar_rotacion, axis=1)

### Codificación y Entrenamiento del modelo

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import joblib

# Codificar variables categóricas
features_encoded = pd.get_dummies(features)

# Codificar la variable objetivo
le = LabelEncoder()
target = le.fit_transform(sales_final['rotacion_categoria'])

# Guardar el encoder
joblib.dump(le, "label_encoder.pkl")

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)

# Entrenar modelo
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Guardar modelo y columnas
joblib.dump(model, "modelo_rotacion.pkl")
joblib.dump(features_encoded.columns.tolist(), "columnas_modelo.pkl")


['columnas_modelo.pkl']

Verificamos el modelo

In [10]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))

              precision    recall  f1-score   support

        Alta       1.00      1.00      1.00        17
        Baja       0.94      0.90      0.92     17431
       Media       0.99      0.99      0.99      4422
  Sobrestock       0.99      0.99      0.99    187845

    accuracy                           0.99    209715
   macro avg       0.98      0.97      0.98    209715
weighted avg       0.99      0.99      0.99    209715



## Streamlit

In [11]:
import streamlit as st
import pandas as pd
import joblib

# Cargar modelo y columnas
model = joblib.load("modelo_rotacion.pkl")
columns = joblib.load("columnas_modelo.pkl")

st.title("Clasificador de Rotación de Productos")

# Inputs del usuario
sales_dollars = st.number_input("Ventas en pesos")
purchase_price = st.number_input("Precio de compra")
volume = st.selectbox("Formato", ['375', '750', '1000'])
month = st.selectbox("Mes", list(range(1,13)))
store = st.selectbox("Tienda", ['Tienda 1', 'Tienda 2', 'Tienda 3'])

# Preparar datos
input_df = pd.DataFrame({
    'SalesDollars': [sales_dollars],
    'PurchasePrice': [purchase_price],
    'Volume': [volume],
    'Month': [month],
    'Store': [store]
})

# Codificar igual que en entrenamiento
input_encoded = pd.get_dummies(input_df).reindex(columns=columns, fill_value=0)

# Predicción
pred = model.predict(input_encoded)
st.write(f"Categoría de rotación estimada: {le.inverse_transform(pred)[0]}")

2025-11-12 13:55:05.618 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-12 13:55:05.923 
  command:

    streamlit run C:\Users\pablo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-11-12 13:55:05.924 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-12 13:55:05.924 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-12 13:55:05.924 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-12 13:55:05.925 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-12 13:55:05.925 Thread 'MainThread': missing ScriptRunContext! This w

In [12]:
import joblib

# Guardar modelo y columnas
joblib.dump(model, "modelo_rotacion.pkl")
joblib.dump(X_train.columns.tolist(), "columnas_modelo.pkl")

['columnas_modelo.pkl']